<a href="https://colab.research.google.com/github/nverchev/Other/blob/main/Chamfer_distance_for_Pytorch_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the best implementation of the Chamfer Distance?

We compare a simple implementation with other two coming from two libraries.
- https://github.com/otaheri/chamfer_distance
- https://pypi.org/project/chamferdist/1.0.0/#description

These implementatations are compatible with torch.autograd, and we test them using GPU power.

I suggest to run the experiment different times to account for some possible overhead.

In [4]:
#First implementation 
!pip install git+'https://github.com/otaheri/chamfer_distance'

#Second implementation
!pip install chamferdist


  Cloning https://github.com/otaheri/chamfer_distance to /tmp/pip-req-build-sx6zoqx4
  Running command git clone -q https://github.com/otaheri/chamfer_distance /tmp/pip-req-build-sx6zoqx4
     |████████████████████████████████| 108 kB 4.3 MB/s 
  Created wheel for chamfer-distance: filename=chamfer_distance-0.1-py3-none-any.whl size=5653 sha256=e88283e85b7b193a2a2fcd7ac6580c3ff68abfea30cb7b73bc1534fa4ec7164e
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ti5ia64/wheels/2a/c5/7c/395771526a57f81590f5b9e2be57f219f834d894e10b1cd993
Successfully built chamfer-distance
  Created wheel for chamferdist: filename=chamferdist-1.0.0-cp37-cp37m-linux_x86_64.whl size=6358578 sha256=1208d8d06bc289545bd956105753c9ac2ab8b8acfe4a2cbc9f83addc5ec2f19e
  Stored in directory: /root/.cache/pip/wheels/28/bb/d1/c789ecd6835e466e813f6e2c5e23bb1bbb2248e84586ba82d2
Successfully built chamferdist


In [1]:
import time
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pc1 = torch.rand([100,1000,3]).to()
pc2 = torch.rand([100,100,3]).to()

In [2]:
from chamfer_distance import ChamferDistance
chamdist = ChamferDistance()
start = time.time()
d1,d2,_,_ = chamdist(pc1,pc2) # dist forward, dist reverse
d = d1.sum(axis=1).mean() + d2.sum(axis=1).mean() # batchmean
end = time.time()
print("First implementation:")
print("Time: ", end - start)
print("Result: ", d)

First implementation:
Time:  0.19299530982971191
Result:  tensor(19.7662)


In [3]:
from chamferdist import ChamferDistance
chamdist = ChamferDistance()
start = time.time()
d = chamdist(pc1,pc2, bidirectional=True, reduction="mean") 
end = time.time()
print("Second implementation:")
print("Time: ", end - start)
print("Result: ", d)

Second implementation:
Time:  0.22507572174072266
Result:  tensor(19.7662)


In [4]:
def square_distance(t1, t2):
    t2 = t2.permute(0, 2, 1)
    dist = -2 * torch.matmul(t1, t2)
    dist += torch.sum(t1 ** 2, 2, keepdim=True)
    dist += torch.sum(t2 ** 2, 1, keepdim=True)
    return dist
    
# Chamfer Distance
def chamdist(t1, t2): 
    dist = square_distance(t1, t2)
    # forward + reverse
    return torch.min(dist, axis = 2)[0].mean(0).sum()\
         + torch.min(dist, axis = 1)[0].mean(0).sum()

start = time.time()
d = chamdist(pc1, pc2)
end = time.time()
print("Simple implementation:")
print("Time: ", end - start)
print("Result: ", d)

Simple implementation:
Time:  0.13100361824035645
Result:  tensor(19.7662)


# Discussion
Surprisingly, the simple implementation in torch is quicker than the ones from the two libraries.

Hope this could be useful to anybody starting with PCD.
